In [ ]:
#pip install langchain langchain_google_vertexai google-cloud-aplatform
import dotenv
import os
from google.cloud import aiplatform
from langchain_google_vertexai import (VertexAIEmbeddings, 
                VectorSearchVectorStore, VectorSearchVectorStoreDatastore, VertexAI)
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain_core.prompts.string import StringPromptTemplate
from langchain.chains import RetrievalQA
import random
import time

In [2]:
#load environment vars
dotenv.load_dotenv()
ProjectId = os.getenv('GOOGLE_VERTEX_PROJECT')
Region = os.getenv('REGION_GOOGLE')
Bucket = os.getenv('BUCKET_GOOGLE')
BucketURI = f"gs://{Bucket}"

In [4]:
aiplatform.init(project=ProjectId, location=Region, staging_bucket=BucketURI)
#setup the endpoint
indexID = os.getenv('INDEXID_GOOGLE')
endpointID = os.getenv('ENPOINTID_GOOGLE')
if indexID == None or endpointID == None:
    displayName = "IndexSecure" + (str)(random.randint(100000, 999999))
    index = aiplatform.MatchingEngineIndex.create_tree_ah_index(display_name=displayName, dimensions=768,
                                                        approximate_neighbors_count=150,
                                                        distance_measure_type="DOT_PRODUCT_DISTANCE",
                                                        index_update_method="STREAM_UPDATE")
    endpoint = aiplatform.MatchingEngineIndexEndpoint.create(display_name=f"{displayName}-endpoint", public_endpoint_enabled=True)
    endpoint = endpoint.deploy_index(index = index, deployed_index_id="DeployedIndex03")

    indexID = endpoint.deployed_indexes[-1].index.split("/")[-1]
    endpointID = endpoint.name
    os.environ['INDEXID_GOOGLE'] = indexID
    os.environ['ENDPOINTID_GOOGLE'] = endpointID
    dotenv.set_key(dotenv_path='.env', key_to_set='INDEXID_GOOGLE', value_to_set=indexID)
    dotenv.set_key(dotenv_path='.env', key_to_set='ENDPOINTID_GOOGLE', value_to_set=endpointID)
else:
    index = aiplatform.MatchingEngineIndex(indexID)
    enpoint = aiplatform.MatchingEngineIndexEndpoint(endpointID)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/158741848249/locations/us-central1/indexes/1901253516517703680/operations/4608412119476469760
MatchingEngineIndex created. Resource name: projects/158741848249/locations/us-central1/indexes/1901253516517703680
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/158741848249/locations/us-central1/indexes/1901253516517703680')
Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/158741848249/locations/us-central1/indexEndpoints/7922935654218989568/operations/4969825989572952064
MatchingEngineIndexEndpoint created. Resource name: projects/158741848249/locations/us-central1/indexEndpoints/7922935654218989568
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/158741848249/locations/us-central1/indexEndpoints/7922935654218989568')
Deploying index Matchi

In [ ]:
#indexID = endpoint.deployed_indexes[-1]
#print(indexID)
indexID = endpoint.deployed_indexes[-1].index.split("/")[-1]
endpointID = endpoint.name
os.environ['INDEXID_GOOGLE'] = indexID
os.environ['ENDPOINTID_GOOGLE'] = endpointID
dotenv.set_key(dotenv_path='.env', key_to_set='INDEXID_GOOGLE', value_to_set=indexID)
dotenv.set_key(dotenv_path='.env', key_to_set='ENDPOINTID_GOOGLE', value_to_set=endpointID)

id: "DeployedIndex01"
index: "projects/158741848249/locations/us-central1/indexes/6023173075468550144"
create_time {
  seconds: 1733023363
  nanos: 390938000
}
index_sync_time {
  seconds: 1733024885
  nanos: 156626000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"



(True, 'ENDPOINTID_GOOGLE', '1159707690373480448')

In [ ]:
#indexID=endpoint.deployed_indexes[-1].index.split("/")[-1]
#print(indexID)

6023173075468550144


In [5]:
embeddingModel = VertexAIEmbeddings(model_name='textembedding-gecko@003')

In [6]:
vectorStorage = VectorSearchVectorStore.from_components(project_id=ProjectId, region=Region, gcs_bucket_name=Bucket,
                                                        index_id=indexID, endpoint_id=endpointID, embedding=embeddingModel, stream_update=True)

In [7]:
textToSearch = [
    "John really likes his pizza",
    "The sky is blue",
    "John's pizza is a circle",
    "John has severe issues",
    "John needs to escape the horrible situtation he is in",
    "John must run until he can escape",
    "Spiralling spiralling spiralling",
    "Oh look a squirell",
    "John dies a brutal death none could have foreseen",
    "Water is important to drink make sure you get enough"
]
vectorStorage.add_texts(texts=textToSearch)

vectorRetrieve = vectorStorage.as_retriever()
vectorRetrieve.search_kwargs = {'k': 1}

Upserting datapoints MatchingEngineIndex index: projects/158741848249/locations/us-central1/indexes/1901253516517703680
MatchingEngineIndex index Upserted datapoints. Resource name: projects/158741848249/locations/us-central1/indexes/1901253516517703680


In [15]:
promptTemplate = """You are an AI assistant given data from documents.
    Answer questions using information provided.
    Do not attempt to make up data you have not been given.
    Simply state "I don't know" if you don't know the answer.
    {summaries}"""

system_message = SystemMessagePromptTemplate.from_template(template=promptTemplate)

human_message = HumanMessagePromptTemplate.from_template(template="{question}")

messages = [
    system_message,
    human_message
]
prompt = ChatPromptTemplate.from_messages(messages)


In [16]:
languageModel = VertexAI(model_name="gemini-1.5-flash")
chain = RetrievalQA.from_chain_type(llm=languageModel, chain_type='stuff', retriever=vectorRetrieve, return_source_documents=True)

In [ ]:
query = "What is the shape of John's pizza?"
result = chain(query)
output_text=f"""Question: 
    {query}
    Answer: 
    {result['result']}
    Source:
    {' '.join(list(set([doc.page_content for doc in result['source_documents']])))}
"""

print(output_text)
#make sure to go into vertex and close the things to avoid spending money on accident

C:\Users\wyatt\AppData\Local\Temp\ipykernel_24976\506884518.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain(query)


Question: 
    What is the shape of John's pizza?
    Answer: 
    A circle. 

    Source:
    John's pizza is a circle

